In [1]:
import csv, os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
All_data = []
for i in range(10):
    All_data.append(pd.read_csv(f'../../preprocess_data/All_data_part{i}.csv', index_col=0, low_memory=False))
    # print(len(All_data[i].columns))
    
All= pd.concat(All_data)

In [3]:
def change_description(value):
    if type(value) == str:
        value = value.lower()
        if 'description' in value:
            return 'description'
    
    return value
    
All.loc[All['why_stopped'].apply(change_description) == 'description','why_stopped'] = All.loc[All['why_stopped'].apply(change_description) == 'description','detailed_description/textblock']

In [4]:
def select_reason(value, reason):
    for r in reason:
        if r==value or r in value:
            return True
        
    return False

In [5]:
PoorEnrollment = ['enroll', 'accrual', 'no participants enrolled', 'insufficient enrollment', 'no participants enrolled', 'poor enrollment rate', 'poor enrolment', 'enrollment difficulties', 
                'difficulty in enrollment', 'no enrollment', 'inadequate enrollment', 'study terminated poor enrollment', 'poor patient enrollment', 'study stopped poor enrollment', 
                'the study was terminated poor enrollment', 'terminated poor enrollment', 'difficulty in recruiting patients', 'no patients enrolled', 'inability to recruit patients',
                'insufficient number of participants', 'poor recrutment', 'poor inclusion', 'no participants', 'because of difficulties in recruting', 'no enrolement', 'due to poor recrutment', 
                'because a significant decrease of viable candidates for the study', 'the study was terminated due to insufficient sample size and protocol compliance issue', 'poor participation',
                'administrative reasons, and did not meet sample size requirements', 'insufficient patients with ckd willing to be injected with heparin', 'no patient enter this study',
                'poor patients', 'number of patients', 'no patients', 'no inclusion', 'difficulties of inclusion', 'inclusion difficulties', 'unrealistic to succeed with sufficient inclusion',
                'inclusion rate not as expected', 'inclusion rate', 'no enough inclusion', 'bad inclusion', 'inclusion criteria difficulties', 'poor new inclusion', 'not able to include enough participants']
efficacy = ['efficacy', 'treatment ineffective', 'poor efficacy', 'no convincing results', 'obtained results were not good. protocol was proved ineffective', 'the study was stopped after the first interim analysis due to futility',
            'the study completed the first phase and did not meet efficacy end point to enter into the\r\n    randomized phase', 
            'the decision to stop the trial was based on efficacy results in the overall schizophrenia\r\n    participant population',
            'futility', ]
safety = ['safety', 'safety concerns', 'toxicity', 'toxicology', 'adverse event', 'due to adverse events', 'increased number of patients death']
Others = ['business', 'funded', ' funds', ' pi ', 'funding', 'no funding', 'funding', 'financial constraints', 'pi left', 'pi leaving', 'pi left institution', 'business reasons', "sponsor's decision",
          'pi left the institution', 'loss of funding', 'insufficient funding', 'funding unavailable', 'administrative', 'covid', 'sponsor terminat', 'poor financial', 'sponsor withdrew',
            'poor funds', 'funding withdrawn', 'funding issues', 'funding ended', 'not funded', 'investigator left institution', 'principal investigator left institution', 'cv-19',
            'study not funded', 'funding stopped', 'strategic business decision', 'pi has left institution', 'business decision', 'funder decision', 'due to poor finace', 'sponsor request',
            'insufficient funds', 'financial issues', 'investigator left', 'sponsor decision', 'microbiological findings of resistance on therapy in patients', 'company decision']

def why_stopped_re(value):
    if type(value) == str:
        value = value.lower()
        value = value.strip().strip('.').replace('accural','accrual').replace('acrual','accrual').replace('recruit', 'enroll').replace('lack of', 'poor').replace('slow', 'low').replace('low', 'poor').replace('enrolment', 'enrollment')
        if 'patient' in value and len(value.split()) < 12: # if the sentence is short and contains patient, it is likely to be poor enrollment
            value = 'poor enrollment' 
        if 'inclusion' in value and len(value.split()) < 12:
            value = 'poor enrollment'
        if 'participant' in value and len(value.split()) < 4:
            value = 'poor enrollment'
        if select_reason(value, Others):
            value = 'Others'
        # if ('enroll' in value) or ('accrual' in value) or ('patient' in value) or ('inclusion' in value) or ('subject' in value) or ('participants' in value):
        #     value = 'poor enrollment'
        if select_reason(value, PoorEnrollment):
            value = 'poor enrollment'
        if select_reason(value, efficacy):
            value = 'efficacy'
        if select_reason(value, safety):
            value = 'safety'
        
    return value

temp = All['why_stopped'].apply(why_stopped_re)

In [6]:
tdf = temp.value_counts()
print(len(tdf))
for i in tdf.index:
    print(i, tdf[i])

11694
poor enrollment 13043
Others 8715
efficacy 1133
safety 920
strategic considerations 37
terminated 37
study never started 37
pi decision 29
investigator decision 28
never started 27
management decision 25
poor resources 24
feasibility 21
withdrawn 20
interim analysis 19
terminated by sponsor 18
the development program has been terminated 17
study terminated by sponsor 15
trial not initiated 15
research cancelled 15
pi no longer at institution 15
logistics 14
development program terminated 14
strategic decision 14
investigator choice 14
corporate decision 14
study not started 13
feasibility issues 13
transition to biometabol 13
pi request 12
study cancelled 12
drug supply issues 12
insufficient staff 12
technical problems 11
competing studies 11
sponsor 11
study was never initiated 11
poor feasibility 11
study was never started 10
study never initiated 10
nonapplicable clinical trial 10
decision of investigational team 10
study completed 10
sponsor r & d strategy adjustment 10
stud

In [7]:
tdf.to_csv('why_stopped.csv')